# 1. Recol·lecció i tractament de dades

 * 1.1 Executem la llibreria LabelMe, que manualment ens permetrà obtenir unes coordenades que marquin la àrea del objecte que volem detectar. Aquesta llibreria ens retornarà un arxiu .json amb els valors numèrics dels punts marcats. Amb aquests valors, la xarxa neuronal serà capaç d'entrenar-se. Haurem de crear una carpeta que anomenarem "labels" on hi aniran aquestes "etiquetes" en format .json

In [ ]:
!labelme #execució de la llibreria labelme

# 2. Creació de la base de dades

In [ ]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
imatges = tf.data.Dataset.list_files('data\\images\\*.png') # introduïm les imatges a la base de dades

In [ ]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.io.decode_png(byte_img)
    return img

In [ ]:
imatges = imatges.map(carregar_imatge) # executem la funció per carregar les imatges en la base de dades 

In [ ]:
imatges.as_numpy_iterator().next() 

### 2.1 Augmentem la quantitat de dades amb la llibreria Albumentation

In [ ]:
import os

In [ ]:
def quantImatgesDividir(dir_carpeta):
    num_elements = len(os.listdir(dir_carpeta))
    n_entrenar = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_provar = round(num_elements*0.15) # 15% de les imatges per provar
    n_validar = num_elements - (n_entrenar + n_provar) # 15% de les imatges per validar
    print(f"{n_entrenar} --> 'train' \n {n_provar} --> 'test' \n {n_validar} --> 'val'")


In [ ]:
quantImatgesDividir('T:\REP\deteccio_prova\data\images') # s'han de dividir manualment les imatges (intentant que sigui aleqatori)

In [ ]:
# mou les els labels a les seves respectives carpetes
for carpeta in ['train','test','val']:
    for arxiu in os.listdir(os.path.join('data', carpeta, 'images')): #per cada arxiu en cada carpeta
        nom_arxiu = arxiu.split('.')[0]+'.json' #extreu el nom de l'arxiu
        dir_existent = os.path.join('data','labels', nom_arxiu)#crea possible direcció
        if os.path.exists(dir_existent): #si la possible direcció existeix, desplaça l'arxiu .json corresponent
            nova_dir = os.path.join('data',carpeta,'labels',nom_arxiu)
            os.replace(dir_existent, nova_dir)#canviem